In [1]:
import os, re, shutil, javalang
import pandas as pd

metrics: 

In [2]:
mt_field = pd.read_csv(r"eclipse_field.csv")
mt_method = pd.read_csv(r"eclipse_method.csv")
mt_file = pd.read_csv(r"eclipse_count_interface_class.csv")

In [3]:
_ = pd.merge(mt_field,mt_method,on=['file'],how= "inner")
mt = pd.merge(_,mt_file,on=['file'],how= "inner")
mt.rename(columns={"file": "pkgFile", "num_of_interface": "NOI", "num_of_class": "NOT", "line_of_code": "TLOC"}, inplace=True)
mt.drop(columns=["num_of_anony"], inplace=True)
mt.head()

,pkgFile,NOF_avg,NOF_max,NOF_total,NSF_avg,NSF_max,NSF_total,NOM_avg,NOM_max,NOM_total,NSM_avg,NSM_max,NSM_total,NOI,NOT,TLOC
0,$packageName$.$actionClass$.java,0.0,0,0,0.0,0,0,3.0,3,3,0.0,0,0,0,1,42
1,$packageName$.$contributorClassName$.java,2.0,2,2,0.0,0,0,6.0,6,6,0.0,0,0,0,1,103
2,$packageName$.$editorClass$.java,1.0,1,1,0.0,0,0,1.0,1,1,0.0,0,0,0,1,20
3,$packageName$.$editorClassName$.java,3.0,3,3,0.0,0,0,16.0,16,16,0.0,0,0,0,1,239
4,$packageName$.$wizardClassName$.java,2.0,2,2,0.0,0,0,8.0,8,8,0.0,0,0,0,1,150


post and pre bug

In [4]:
def count_bug(bug_file, version):
    bug_info =  pd.read_csv(bug_file)
    _ = bug_info[["file", "bugID", "releaseNo"]]
    post_version = _[_["releaseNo"] == version]
    s = post_version["file"].value_counts()
    bug_num = pd.DataFrame(
        {
            "file": s.index, 
            "num_of_bug": s.values
        }
    )
    return bug_num

post_bug = count_bug(r"eclipse_post_bug.csv", "R3_0_1")
pre_bug = count_bug(r"eclipse_pre_bug.csv", "R3_0_1")

In [5]:
post_bug.head()

,file,num_of_bug
0,eclipse\e4\org.eclipse.e4.resources\bundles\or...,1
1,eclipse\e4\org.eclipse.e4.resources\bundles\or...,1
2,eclipse\e4\org.eclipse.e4.resources\bundles\or...,1
3,eclipse\org.eclipse.jdt.core.tests.model\works...,1
4,eclipse\org.eclipse.jdt.ui\ui\org\eclipse\jdt\...,1


merge them

In [6]:
def generate_dataset(bugTable, metricsTable, map_rst_path):
    file_mapping = pd.read_csv(map_rst_path)
    _ = pd.merge(metricsTable, file_mapping, on=['pkgFile'], how= "inner")
    return pd.merge(_, bugTable, on=['file'], how= "left")

In [7]:
dataset_post = generate_dataset(post_bug, mt, r"eclipse_map_filePath_packageFileName.csv")
dataset_pre = generate_dataset(pre_bug, mt, r"eclipse_map_filePath_packageFileName.csv")

In [8]:
dataset_post.head()

,pkgFile,NOF_avg,NOF_max,NOF_total,NSF_avg,NSF_max,NSF_total,NOM_avg,NOM_max,NOM_total,NSM_avg,NSM_max,NSM_total,NOI,NOT,TLOC,file,num_of_bug
0,org.eclipse.ant.internal.ui.console.JavacLineT...,6.0,6,6,0.0,0,0,3.0,3,3,0.0,0,0,0,1,163,eclipse\org.eclipse.ant.ui\Ant Tools Support\o...,1
1,org.eclipse.ant.internal.ui.editor.model.AntDe...,0.0,0,0,0.0,0,0,4.0,4,4,1.0,1,1,0,1,105,eclipse\org.eclipse.ant.ui\Ant Editor\org\ecli...,1
2,org.eclipse.ant.internal.ui.editor.text.AntEdi...,0.0,0,0,2.0,2,2,0.0,0,0,1.0,1,1,0,1,31,eclipse\org.eclipse.ant.ui\Ant Editor\org\ecli...,1
3,org.eclipse.ant.internal.ui.editor.text.FileSc...,0.0,0,0,0.0,0,0,2.0,2,2,0.0,0,0,0,1,26,eclipse\org.eclipse.ant.ui\Ant Editor\org\ecli...,1
4,org.eclipse.ant.internal.ui.editor.text.XMLTex...,1.0,1,1,0.0,0,0,9.0,9,9,1.0,1,1,0,1,254,eclipse\org.eclipse.ant.ui\Ant Editor\org\ecli...,1


In [10]:
dataset_post.to_csv("dataset-eclipse_3.0.1-filelevel-post.csv", index=False)
dataset_pre.to_csv("dataset-eclipse_3.0.1-filelevel-pre.csv", index=False)

In [5]:
file_mapping = pd.read_csv(r"eclipse_map_filePath_packageFileName.csv")
a1 = file_mapping["pkgFile"].value_counts()
hehe = pd.DataFrame(
    {
        "file": a1.index, 
        "num_of_bug": a1.values
    }
)
hehe.to_csv("hehe.csv", index= False)

In [8]:
exp = file_mapping[file_mapping["pkgFile"] == "org.eclipse.core.runtime.internal.adaptor.Locker_JavaIo.java"]
exp.to_csv("hehe1.csv", index=False)